In [27]:
import numpy as np 
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from LSTM.PytorchLSTMTrain import LSTMModel
from Transformer.autoregressiveTrain import MotorTransformer 
from IPython.display import display
import ipywidgets as widgets
import time


In [28]:
#useful functions 
def train(model,loader,n_epochs,start_channel,end_channel,window,sched,filename):
    scheduler=np.arange(0,0.9,0.9/(n_epochs))
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    history=[]
    label = widgets.Label(value="Starting…")
    display(label)
    for epoch in range(n_epochs):
        total_loss = 0.0
        label.value="Live Loss: ..."
        for (X,) in loader:     # X: (batch, seq_len, input_dim)
            X = X.to(device)
            optimizer.zero_grad()
            seq_len=X.shape[1]
            hidden = None                   # LSTM hidden state
            losses = []
            #predict each future step autoregressively
            inp = X[:, :window,start_channel:end_channel]
            for t in range(window, seq_len):
                # Forward pass
                pred=model(inp)  # (batch,1,12)

                # Compute target delta
                target= X[:, t-window+1:t+1, :12]
                #target_delta = X[:, t-window+1:t+1, :12] - X[:, t-window:t, :12]
                #target_delta = target_delta.unsqueeze(1)  # (batch,1,12)
                losses.append(criterion(pred, target))

                # Reconstruct absolute motor positions
                motors = pred#inp[:, :, :12] + pred
                # Scheduled sampling
                if np.random.random() < scheduler[epoch] and sched:
                    if start_channel<12:
                        remainder = X[:, t-window+1:t+1, 12:end_channel]
                        inp = torch.cat([motors.detach(), remainder.to(device)], dim=2)
                    else: inp= X[:, t-window+1:t+1, start_channel:end_channel]
                else:
                    inp = X[:, t-window+1:t+1, start_channel:end_channel]


            loss = torch.stack(losses).mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            total_loss += loss.item()
            clear_line_up()
            label.value=f"Live Loss: {total_loss:.6f}"
        avg_loss = total_loss / len(loader)
        label.value=f"Epoch {epoch+1:02d} | Loss: {avg_loss:.6f} | Probability:"+str(scheduler[epoch]*sched)
        history.append(avg_loss)
        torch.save(model.state_dict(), filename+"_model")
    history=np.array(history)
    np.save(filename+"loss",history)
    return history

## Loss
Train a bunch of models normally and save the loss, then average and show 

In [29]:
X=np.load("/its/home/drs25/Quadruped/Code/UBERMODEL/models/X_sorted.npy")
X_tensor = torch.tensor(X, dtype=torch.float32)
dataset = TensorDataset(X_tensor)
WINDOW=15
for trial in range(20):
    loader = DataLoader(dataset, batch_size=128, shuffle=True)
    INPUT_DIM=X_tensor.shape[2]
    OUTPUT_DIM=12
    model = LSTMModel(input_dim=INPUT_DIM, output_dim=OUTPUT_DIM).to(device)
    train(model,loader,500,0,20,WINDOW,1,"/its/home/drs25/Quadruped/Code/UBERMODEL/models/experiments/loss/LSTMtrial_"+str(trial))

for trial in range(20):
    loader = DataLoader(dataset, batch_size=128, shuffle=True)
    INPUT_DIM=X_tensor.shape[2]
    OUTPUT_DIM=12
    model = MotorTransformer(input_dim=INPUT_DIM,output_dim=OUTPUT_DIM,T=WINDOW).to(device)
    train(model,loader,500,0,20,WINDOW,1,"/its/home/drs25/Quadruped/Code/UBERMODEL/models/experiments/loss/TRANStrial_"+str(trial))

Label(value='Starting…')

Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...
Live Loss: ...


KeyboardInterrupt: 

In [ ]:
for trial in range(20):
    f1="/its/home/drs25/Quadruped/Code/UBERMODEL/models/experiments/loss/LSTMtrial_"+str(trial)
    f2="/its/home/drs25/Quadruped/Code/UBERMODEL/models/experiments/loss/TRANStrial_"+str(trial)

## Useful data
Train various models on different parts of the data 
- motors only
- all signals
- tactile vectors and friction only
Plot the data in a table 

## Dataset size 
Look at model performance depending on dataset size and diversity of dataset

## SImulation performance
Pick the best parameters from above and look at trajectoris and distance as metrics 

## Stat testing 
FInally stat test the model performances of the best paramters from the above experiments